# ISS_preprocessing of Leica .lif files

This notebook guides you through the preprocessing of .lif files by Leica microscopes. **The sofware is not able and it is not designed to process .lof files.**  

To use this notebook you must saved directly from the Leica software, a single .lif file per cycle. 

Each file can contain multiple regions, but these need to be consistent across the various cycles.

You need to manually specify which cycle the file refers to, as in the example below.

We begin importing the necessary libraries and tools

In [4]:
import ISS_processing.preprocessing as pp
import os
import pandas as pd

### `lif_mipping`

This function extracts the images, organises them, and create maximum projections that are exported with a naming convention fitting the downstream processing steps.

It also parses the metadata and converts them for downstream processing.


`lif_mipping` takes as inputs the following arguments:

`lif_path`: the path to the lif file that you want to preprocess, down to the lif file (included)

`output_folder`: the folder where you want to save the maximum-projected images. 

`cycle`: here you have to manually specify to which ISS cycle the images refer to. This is a `int` number, where 1 refers to cycle 1 and so on. 

In [ ]:
from ISS_processing import preprocessing 
lif_path='/path/to/my/cycle1/file.lif'
output_dir = '/path/to/my/output/folder/'

In [ ]:

lif_mipping(lif_path, output_dir, cycle=1)

### `leica_OME_tiff`
`leica_OME_tiff` is the function that **takes the projected images across channels and wraps them into a single OMEtiff per imaging cycle**. This steps organises the files corresponding to each imaging cycles in a specific way within a single file and requires the parsing of a Metadata file to arrange correctly the images in xy space. As explained before, the input for this function is the specific sub-folder within `output_location` one wishes to process. **This function does not allow to process multiple regions in one go, and needs to be run on individual regions manually**.

The function outputs 1 OMEtiff file per cycle into the `/preprocessing/OME_tiffs/` subfolder.



In [ ]:
from ISS_processing import preprocessing 

path = '/path/to/output_folder/region/'
# create leica OME_tiffs
pp.leica_OME_tiff(
    directory_base = path+'/preprocessing/mipped/',
    output_directory = path+'/preprocessing/OME_tiffs/'
)

### `ashlar_wrapper`

This function runs `ashlar`, a package for image stitching and cycle alignment. The function uses the OME_tiffs files as an input, takes as input a channel number (normally the DAPI, see above) and on that channels performs all the alignment and stitching operations.

The function outputs 1 stitched file per cycle and channel into the `/preprocessing/stitched/` subfolder

In [ ]:
from ISS_processing import preprocessing 

# align and stitch images
path = '/path/to/output_folder/region/'
OME_tiffs = os.listdir(path+'/preprocessing/OME_tiffs/')
OME_tiffs = [path+'/preprocessing/OME_tiffs/' + sub for sub in OME_tiffs]

pp.ashlar_wrapper(
    files = OME_tiffs,
    output = path+'/preprocessing/stitched/',
    align_channel=4
)


### `tile_stitched_images`
In this function the stitched images are re-tiled according to a user-specified size.
The reason for this is that stitched images are too big to be decoded directly and we prefer to decode them in tiles. This has several advantages, most notably that the pipeline would work also on laptops or non-powerful computers. The idea tile size is 4000-6000, but larger or smaller are also fine depending on the computer.

In [ ]:
from ISS_processing import preprocessing 

# align and stitch images
path = '/path/to/output_folder/region/'

# retile stitched images
pp.tile_stitched_images(
    image_path = path+'/preprocessing/stitched/',
    outpath = path+'/preprocessing/ReslicedTiles/',
    tile_dim=2000
)

